In [ ]:
!pip install transformers

I Use my own model for this part you can find it in its own repository in my github, download it and put its path here

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Define the path to your model
model_path = ""

# Load the model and tokenizer
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
context = """The Great Wall of China is one of the most famous landmarks in the world.
 It stretches over 13,000 miles and was originally built to protect against invasions from nomadic tribes.
  Construction of the wall began in the 7th century BC and continued until the 16th century.
   It was made primarily of earth, wood, and stone.
    The wall also served as a means of border control and trade route security.
     During the Ming dynasty, the wall was strengthened and expanded to its current form.
      Today, it is a UNESCO World Heritage Site and a symbol of China's historical strength and ingenuity.
       Visitors from around the globe come to see its impressive structure and learn about its history.
        The wall's construction involved millions of workers, including soldiers, peasants, and prisoners."""


**Fact Based Questions:**

In [ ]:
ANSWER_K = 30
ANSWER_P = 0.5
ANSWER_TEMP = 0.6

In [ ]:
def generate_fact_based_answer(context, question,
                               temperature=ANSWER_TEMP,
                               top_k=ANSWER_K,
                               top_p=ANSWER_P):
    # Prepare the input text
    prompt ="""as a Teacher, Based on the provided context,
     synthesize the information to answer using full sentences:"""
    input_text = f"{prompt} question: {question} context: {context} "
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Generate the answer
    outputs = model.generate(
        input_ids,
        num_beams =6,
        max_length=50,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        num_return_sequences=1,
        no_repeat_ngram_size=3,
        do_sample = True,
        repetition_penalty=3.0,
        length_penalty=1.5
    )
    # Decode and return the answer
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [ ]:
import itertools

temp_arr_low = [0.5, 0.6, 0.7]
temp_arr_moderate = [0.8, 0.9, 1]
temp_arr_high = [1.2, 1.3]

k_arr_low = [10, 25, 40]
k_arr_moderate = [50, 70, 100]
k_arr_high = [110, 150, 200]

p_arr_low = [0.5, 0.55, 0.6, 0.65]
p_arr_moderate = [0.7, 0.75, 0.8, 0.85]
p_arr_high = [0.9, 0.95, 1]


combinations = list(itertools.product(temp_arr_high, k_arr_high, p_arr_high))

questions = [
    "What were the primary materials used in the construction of the Great Wall of China?",
    "During which dynasty was the Great Wall of China strengthened and expanded to its current form?",
    "What was the original purpose of the Great Wall of China?",
    "How many miles does the Great Wall of China stretch over?",
    "Who were involved in the construction of the Great Wall of China?"
]

for temp, k, p in combinations:
  print(f"temp: {temp}, top_k= {k}, top_p= {p}")
  for question in questions:
    print(question)
    print(generate_fact_based_answer(context, question, temperature=temp, top_k=k, top_p=p))
    print()

temp: 1.2, top_k= 110, top_p= 0.9
What were the primary materials used in the construction of the Great Wall of China?
earth, wood, and stone

During which dynasty was the Great Wall of China strengthened and expanded to its current form?
the Ming dynasty

What was the original purpose of the Great Wall of China?
protect against invasions from nomadic tribes

How many miles does the Great Wall of China stretch over?
13,000

Who were involved in the construction of the Great Wall of China?
millions of workers, including soldiers, peasants, and prisoners

temp: 1.2, top_k= 110, top_p= 0.95
What were the primary materials used in the construction of the Great Wall of China?
earth, wood, and stone

During which dynasty was the Great Wall of China strengthened and expanded to its current form?
the Ming dynasty

What was the original purpose of the Great Wall of China?


KeyboardInterrupt: 

**Multiple Choices Questions**

**True or False Questions**